# Crossmatch habitable zone targets with the CKS sample, Gaia DR2 and my own rotation period catalogue.

In [84]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from astropy.io import fits
from astropy.table import Table

Load Stephen Kane's list of targets, the CKS catalogue, the McQuillan catalogue, my own rotation period catalogue and Gaia DR2.

In [85]:
# Kane catalogue:
kane = pd.read_csv("kane_targets.csv")

# CKS catalogue:
cks = pd.read_csv("cks_physical_merged.csv")

# McQuillan catalogue:
mc = pd.read_csv("Table_1_Periodic.txt")

# Tim and my catalogue:
tdmra = pd.read_csv("koi_periods_0712.csv")

# Gaia DR2
dat = Table.read('kepler_dr2_4arcsec_updated.fits', format='fits')
dr2 = dat.to_pandas()

What's the overlap between Stephen Kane's catalogue and the CKS sample?

In [86]:
# Format the CKS catalogue and Kane catalogue column headers.
koi_ids = []
for i in cks.id_koicand:
    koi_ids.append("Kepler-{}".format(int(float(i[1:]))))
cks["koi_id"] = koi_ids

KOIs = np.zeros(np.shape(kane)[0], dtype=int)
for j, i in enumerate(kane.koi_id):
    if i[:7] == "Kepler-":
        KOIs[j] = int((float(i[7:])))

kane["KOI"] = KOIs
print(kane.keys(), "\n")
print(cks.keys())
kane_cks = pd.merge(kane, cks, on="koi_id", how="inner", suffixes=["_KIC", "_CKS"])
print(np.shape(kane), np.shape(cks), np.shape(kane_cks))

Index(['koi_id', 'planet_designation', 'p_orb_days', 'a_AU', 'r_pl_earth',
       'teff', 'r_star_rsol', 'insolation_flux_F_E', 'KOI'],
      dtype='object') 

Index(['id_starname', 'id_kic', 'id_koicand', 'id_kepler_name',
       'koi_disposition', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0', 'koi_time0_err1',
       ...
       'iso_sma', 'iso_sma_err1', 'iso_sma_err2', 'iso_insol',
       'iso_insol_err1', 'iso_insol_err2', 'iso_teq', 'iso_teq_err1',
       'iso_teq_err2', 'koi_id'],
      dtype='object', length=121)
(1162, 9) (2025, 121) (617, 129)


617 stars have CKS spectra. ,How many of these stars have rotation periods measured by Amy McQuillan?

In [87]:
kane_cks_mc = pd.merge(kane_cks, mc, left_on="id_kic", right_on="kepid", how="inner", suffixes=["", "_mc"])
np.shape(kane_cks_mc)

(0, 140)

None... How about in Tim and my catalogue?

In [88]:
kane_cks_tdmra = pd.merge(kane_cks, tdmra, on="KOI", how="inner", suffixes=["", "_tdmra"])
np.shape(kane_cks_tdmra)

(552, 142)

How many of these have Gaia DR2 distances?

In [81]:
# Make cuts on parallax signal-to-noise
m = 10 < dr2.parallax_over_error.values  # Signal to noise greater than 10
gdr2 = dr2.iloc[m]

gdr2.kepid.values[0]
int_kepid = []
for i in gdr2.kepid.values:
    int_kepid.append(int(i))
gdr2["int_kepid"] = np.array(int_kepid, dtype=int)

kane_cks_tdmra_dr2 = pd.merge(kane_cks_tdmra, gdr2, left_on="id_kic", right_on="int_kepid", how="inner")
print(np.shape(kane_cks_tdmra_dr2))

kane_cks_tdmra_dr2.to_csv("kane_cks_tdmra_dr2.csv")

/Users/ruthangus/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(546, 244)


So there are 546 stars with spectroscopic properties, distances AND rotation periods. The next thing to do is to combine all this information using isochrones and a gyrochronology model. See the ipython notebook called age_inference.ipynb for that.